# Tarea 4: Procesamiento de Audio

## Análisis Avanzado de Audio con Python

In [ ]:
# Importar librerías necesarias
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq


## Carga del Archivo de Audio

In [ ]:
# Cargar archivo de audio estéreo
file_path = 'game_of_thrones.wav'
sr_stereo, stereo_audio = wavfile.read(file_path)

# Mostrar características del audio estéreo
print("Características del Audio Estéreo:")
print(f"Frecuencia de muestreo: {sr_stereo} Hz")
print(f"Número de canales: {stereo_audio.shape[1] if len(stereo_audio.shape) > 1 else 1}")
print(f"Tamaño del archivo: {os.path.getsize(file_path) / (1024*1024):.2f} Mb")

# Convertir a mono (promediando canales)
if len(stereo_audio.shape) > 1:
    mono_audio = np.mean(stereo_audio, axis=1).astype(stereo_audio.dtype)
else:
    mono_audio = stereo_audio

# Mostrar características del audio mono
print("\nCaracterísticas del Audio Mono:")
print(f"Frecuencia de muestreo: {sr_stereo} Hz")
print(f"Número de canales: 1")

## Gráfica en el Dominio del Tiempo

In [ ]:
# Gráficas en el dominio del tiempo
plt.figure(figsize=(12, 6))

# Gráfica de audio estéreo
plt.subplot(2, 1, 1)
plt.title('Señal de Audio Estéreo')
if len(stereo_audio.shape) > 1:
    time_stereo = np.linspace(0, len(stereo_audio)/sr_stereo, len(stereo_audio))
    plt.plot(time_stereo, stereo_audio[:, 0], label='Canal Izquierdo')
    plt.plot(time_stereo, stereo_audio[:, 1], label='Canal Derecho')
else:
    time_stereo = np.linspace(0, len(stereo_audio)/sr_stereo, len(stereo_audio))
    plt.plot(time_stereo, stereo_audio)
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.legend()

# Gráfica de audio mono
plt.subplot(2, 1, 2)
plt.title('Señal de Audio Mono')
time_mono = np.linspace(0, len(mono_audio)/sr_stereo, len(mono_audio))
plt.plot(time_mono, mono_audio)
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')

plt.tight_layout()
plt.show()

## Conceptos Técnicos de Audio

Conceptos Técnicos de Audio:
1. **Frecuencia de muestreo:** Número de muestras tomadas por segundo (aquí es 44100 Hz)
2. **Aliasing:** Distorsión que ocurre cuando la frecuencia de muestreo es insuficiente para representar la señal original
3. **Profundidad de bits:** Número de bits usados para representar cada muestra de audio
4. **Ancho de banda:** Rango de frecuencias que puede transmitir una señal
5. **Tasa de bits:** Cantidad de datos procesados por unidad de tiempo

## Transformada Rápida de Fourier (FFT)

In [ ]:
# La longitud del array de datos y el
# sample rate (frecuencia de muestreo).
n = len(mono_audio)
Fs = sr_stereo


# Calculando la Transformada Rapida de Fourier (FFT) en audio mono.
ch_Fourier = np.fft.fft(mono_audio)  # ch1

# Solo miramos frecuencia por debajo de Fs/2
# (Nyquist-Shannon) --> Spectrum.
abs_ch_Fourier = np.absolute(ch_Fourier[:n//2])

# Graficamos.
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()

## Explicación de la FFT

1. **Concentración de energía en bajas frecuencias**  
   - La gráfica muestra un pico muy alto cerca de 0 Hz (DC o muy bajas frecuencias).  
   - En muchos audios o señales, gran parte de la energía se concentra en componentes graves o bajas frecuencias.

2. **Decaimiento rápido**  
   - Tras el pico inicial, la amplitud cae de forma pronunciada, indicando que las frecuencias medias y altas tienen menor energía.  
   - Esto puede deberse a la naturaleza de la señal (por ejemplo, voz, música con predominio de frecuencias graves o una fuerte componente de baja frecuencia).

3. **Escala de amplitud**  
   - El eje y alcanza valores del orden de 10^8, lo cual sugiere que la señal podría tener una amplitud significativa o un offset (DC).  
   - A partir de cierto punto, la amplitud en frecuencias más altas es casi nula, reflejando poca presencia de energía en esas bandas.

En conjunto, la FFT muestra que la señal está dominada por componentes de baja frecuencia, con un gran pico inicial y un rápido decaimiento hacia frecuencias más altas.

## Cálculo de la energía del espectrograma y frecuencia de corte

In [ ]:
# Definimos diferentes epsilons: la parte de
# la energia del espectro que NO conservamos.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

# Jugamos con los valores de epsilon (Variar para ver en gráfica).
eps = eps[5]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el
# valor de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

## Comentario sobre la FFT y la línea de corte (ε = 0.101)

1. **Distribución de la energía**  
   - El espectro presenta un pico muy grande en bajas frecuencias y decae rápidamente. Esto indica que la mayor parte de la energía de la señal se concentra en frecuencias graves o cercanas a 0 Hz.

2. **Línea roja de corte**  
   - La línea roja representa la frecuencia \( f_0 \) donde se conserva \( 1 - \varepsilon \approx 89.9\% \) de la energía total (dado que \(\varepsilon = 0.101\)).  
   - Por debajo de esa frecuencia, se acumula casi toda la energía (un 89.9%), mientras que por encima de ella solo queda el 10.1%.

3. **Interpretación**  
   - En este caso, la línea aparece relativamente baja en frecuencia (unos pocos kHz), confirmando que, tras acumular el 89.9% de la energía, la contribución de las frecuencias más altas es muy pequeña.  
   - Esta alta concentración de energía en la banda baja es típica en ciertas señales de audio o señales con un contenido predominante en graves.


## Compresión de Audio

In [ ]:
# Calculamos el factor D de downsampling.
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos (slicing with stride).
new_data = mono_audio[::D]

# Definimos el nombre del audio comprimido generado y su path.
wav_compressed_file = "game_of_thrones_compressed.wav"
audio_output_path = './'

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path , wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

# Cargamos el nuevo archivo.
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(audio_output_path, wav_compressed_file))

Este factor depende directamente por tanto de la frecuencia de corte f0, que a su vez depende del valor de epsilon. Por tanto, si epsilon es muy pequeño, f0 será muy grande y D será muy pequeño, lo que implica que la señal comprimida tendrá una frecuencia de muestreo muy alta. Por otro lado, si epsilon es muy grande, f0 será muy pequeño y D será muy grande, lo que implica que la señal comprimida tendrá una frecuencia de muestreo muy baja.

## Comparación de Espectrogramas

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(mono_audio, NFFT=1024, Fs=sr_stereo, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

Diferencias entre los espectrogramas

- **Rango de frecuencias**: El original llega hasta ~20 kHz; el comprimido se corta alrededor de ~10 kHz.
- **Pérdida de frecuencias altas**: Se eliminan componentes por encima de 10 kHz, reduciendo brillo/detalles.
- **Resolución espectral**: Al comprimir o bajar la tasa de muestreo, se reduce la nitidez en altas frecuencias.


## Comparación de Tamaños de Archivos

In [ ]:
# Comparación de tamaños de archivos
original_size = os.path.getsize(file_path) / (1024*1024)
compressed_size = os.path.getsize(audio_output_path) / (1024*1024)

print(f"\nTamaño del archivo original: {original_size:.2f} Mb")
print(f"Tamaño del archivo comprimido: {compressed_size:.2f} Mb")

## Reproducción de Audios

In [ ]:
# Reproducir los audios con widgets de IPython
print("Audio original:")
IPython.display.Audio("game_of_thrones.wav", rate=sr_stereo)


In [ ]:
print("Audio comprimido:")
IPython.display.Audio("game_of_thrones_compressed.wav", rate=new_sample_rate)